Esse arquivo tem como intuito tratar, 'plotar' e avaliar os datasets presentes a pasta Saneamento/datasets com o fim de compor a base metodológica do trabalho semestral de RP2 "Uma análise quantitativa da distribuição de casos de dengue na cidade de São Paulo em suas várias granularidades."

Para melhor aproveitamento desse arquivo use Python 3.

Logo abaixo, unimos os datasets (oriundos do IBGE) para facilitar a manipulação e tratamento.

In [ ]:
# Importar a biblioteca pandas
import pandas as pd

# Carregar os três datasets
df1 = pd.read_csv('dataset1.csv')
df2 = pd.read_csv('dataset2.csv')
df3 = pd.read_csv('dataset3.csv')

# Verificar as primeiras linhas para garantir que a coluna ID está correta
print("Dataset 1:")
print(df1.head())
print("\nDataset 2:")
print(df2.head())
print("\nDataset 3:")
print(df3.head())

# Supondo que a primeira coluna se chama 'ID' em todos os datasets
# Unir df1 e df2 com base na coluna 'ID'
merged_df = pd.merge(df1, df2, on='ID', how='inner')

# Unir o resultado com df3
final_df = pd.merge(merged_df, df3, on='ID', how='inner')

# Verificar o dataset
print("\nDataset:")
print(final_df.head())

# (Opcional) Salvar o dataset combinado
final_df.to_csv('dataset_combinado.csv', index=False)
# ================================
# Eliminação de algumas variáveis: as que tratam do gênero dos ocupantes do domicílio, algo que não será investigado
# Definindo os intervalos de início e fim
intervalos = [
    (0, 9),
    (48, 54),
    (112, 144),
    (200, 203),
    (310, 318),
    (398, 404),
    (464, 472),
    (509, 517),
    (541, 544),
    (581, 589),
    (613, 619),
    (637, 641)
]

# gera a lista de índices de colunas a manter
colunas_manter = [i for start, end in intervalos for i in range(start, end)]

# seleciona apenas essas colunas (por posição)
final_df = final_df.iloc[:, colunas_manter]

# (Opcional) Salvar o dataset processado e verificar novamente as informações do dataset
final_df.to_csv('dataset_combinado.csv', index=False)
print("\nInformações do dataset combinado:")
print(final_df.info())

Aplicação das diretrizes de tratamento (remoção de nulos, inválidos, identificação de outliers, normalização e plotagem)

In [ ]:
#CODE FOR FILTERS

In [17]:
# Limpeza de Dados: Converter V00... columns para numérico, tratando 'X' como 0.
# Identificar as colunas a serem processadas (todas as colunas, exceto 'ID')
cols_to_convert = final_df.columns.drop('ID')

# Substituição de 'X' (código de supressão) por 0 nas colunas de variáveis
final_df[cols_to_convert] = final_df[cols_to_convert].replace('X', 0)

# Conversão de todas as colunas de variáveis para o tipo numérico (inteiro)
# O 'errors=\'coerce\'' garante que qualquer outro valor não numérico seja transformado em NaN,
# que é subsequentemente preenchido com 0 por segurança.
final_df[cols_to_convert] = final_df[cols_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# Extração do Código do Distrito (ID_Distrito)
# O código do setor censitário (ID) tem 15 dígitos.
# O código do Distrito (DD) corresponde aos dígitos na posição 8 e 9 (índice 7 e 8, começando em 0).
final_df['ID_str'] = final_df['ID'].astype(str).str.zfill(15)
final_df['ID_Distrito'] = final_df['ID_str'].str[7:9]

# Agregação: Agrupar por ID_Distrito e somar as variáveis V00...
# As variáveis de contagem do Censo devem ser somadas para agregar em um nível territorial maior.
# Removemos as colunas 'ID_str' e 'ID' que não devem ser somadas.
df_grouped = final_df.drop(columns=['ID_str', 'ID']).groupby('ID_Distrito').sum()

# Resetar o índice para que 'ID_Distrito' volte a ser uma coluna de dados.
df_grouped = df_grouped.reset_index()
df_grouped['ID_Distrito'] = df_grouped['ID_Distrito'].astype(int)

# Dataset com informações sobre os distritos - nome, área
df_nomes_distritos = pd.read_csv('Documents/codigos_distritos_msp.csv', sep=';')

# Juntar os dois DataFrames pela coluna ID_Distrito
df_final = pd.merge(
    df_grouped,
    df_nomes_distritos,
    on='ID_Distrito',
    how='left'  # Usa 'left' para manter todos os distritos do seu dataset de saneamento.
)
colunas_principais = ['ID_Distrito', 'Nome']

## Obter todas as outras colunas (removendo as colunas duplicadas do merge e as principais)
colunas_restantes = [
    col for col in df_final.columns
    if col not in colunas_principais and not col.endswith('_x') and not col.endswith('_y')
]

## Criar a nova ordem da coluna
nova_ordem_colunas = colunas_principais + colunas_restantes

## Aplicar a nova ordem ao DataFrame
df_final = df_final[nova_ordem_colunas]

# Salvar o arquivo resultante
df_final.to_csv('dataset_agrupado_por_distrito.csv', index=False)
print("\nInformações do dataset final:")
print(df_final.info())

# Exibir informações de confirmação
print("Dados Agrupados por Distrito (Primeiras Linhas):")
print(df_final.head())

Avaliação